# Knochentypen

Zuerst gruppieren wir die Daten nach Knochentyp, weil wir nur innerhalb dieser Gruppen valide Vergleiche machen können.

In [8]:
import utils.load_write as lw
import utils.datastructure_operations as dataops

df_horse = lw.load_csv("data/csv/abmap_horse.csv")

element_grp = df_horse.groupby(['ELEMENT'])
#print(df.ELEMENT.nunique(), "distinkte Einträge")
print("Es existieren Funde zu", element_grp.ngroups, "verschiedenen Knochentypen.")
list(element_grp.groups.keys())

Es existieren Funde zu 14 verschiedenen Knochentypen.


['Astragalus',
 'Calcaneum',
 'Femur',
 'First phalanx',
 'Humerus',
 'Mandible',
 'Metacarpal',
 'Metatarsal',
 'Pelvis',
 'Radius',
 'Scapula',
 'Second phalanx',
 'Tibia',
 'Ulna']

Nun aktualisieren wir für jeden Datenpunkt die jeweilige Datierung (Periode), sodass wir nur noch mit den vier zuvor festgelegten, gröberen Perioden arbeiten. Dann zählen wir, aus wie vielen dieser Perioden für jeden Typ Funde vorliegen, denn wenn nur Funde aus einer Periode vorliegen, eignet sich der entsprechende Knochentyp natürlich nicht für einen Vergleich über die Zeit.
Außerdem zählen wir, wie viele verschiedene Knochen jeweils gefunden wurden und wie viele Messungen pro Typ gemacht wurden, denn auch hier gilt: Zu wenige Daten liefern keine aussagekräftigen Ergebnisse.

In [ ]:
import pandas as pd
time_ranges_condensed = lw.load_json("data/json/time-ranges-condensed.json")

bone_types = dict()

for bone_type in list(element_grp.groups.keys()):
    ''' 'PERIOD' updaten und alle Spalten entfernen, die nicht benötigt werden '''
    element_dataframe = dataops.update_period(element_grp.get_group(bone_type), time_ranges_condensed)
    element_dimension_grp = element_dataframe.groupby('MEASTYPE')

    ''' save the groups to files for better overview '''
    #path = "data/csv/elements/" + bone_type.lower().replace(" ","_")
    #lw.save_csv(element_dataframe, path + ".csv")
    #lw.save_dict_to_csv(dataops.create_dataframe_from_groups(element_dimension_grp), path + "/")

    ''' zählen, wie viele verschiedene Knochen gefunden wurden '''
    ctr = len(dataops.count_amounts_in_column(element_dataframe, "BONEID"))

    ''' zählen, wie viele verschiedene Perioden enthalten sind '''
    periods = len(dataops.count_amounts_in_column(element_dataframe, "PERIOD"))

    bone_types.update(
        {
            bone_type : {
                "df" : element_dataframe,
                "bone_amt": ctr,
                "meas_amt" : len(element_dataframe),
                "periods_amt": periods
            }
        }
    )

Die Anzahl der Funde und Messungen pro Knochentyp und auch pro Periode sollen als Barchart abgebildet werden.
Zur einfacheren Weiterverarbeitung werden diese Daten daher in Listen gespeichert.

In [ ]:
''' Lists for barchart data '''
bone_amounts = []
meas_amounts = []

df_bone_types = pd.DataFrame(columns=['BONEID', 'MEASURE', 'MEASTYPE', 'PERIOD', 'RANGE'])

for key in list(bone_types.keys()):
    bones = bone_types[key]["bone_amt"]
    bone_amounts.append(bones)
    meas = bone_types[key]["meas_amt"]
    meas_amounts.append(meas)
    periods = bone_types[key]["periods_amt"]

    print(key, ":", bones, "Fund(e) und", meas, "Messungen. Datierungen für", periods, "Perioden.")

    ''' Remove Types that don't have enough data to support any statistics about them '''
    if bones < 10 or periods <= 2:
        print("- DELETING", key, " - not enough data")
        del bone_types[key]
    else:
        df_bone_types = pd.concat([df_bone_types, bone_types[key]["df"]])

Die Einträge mit weniger als 10 Funden und/oder weniger als 2 Perioden werden in der weiteren Betrachtung außer Acht gelassen, da sie nicht ausreichend Daten für einen Vergleich liefern. Dies betrifft die Einträge für "Mandible", "Second phalanx" und "Ulna".
Die Anzahl an Knochentypen reduziert sich damit von 14 auf 11.

Nächster Schritt: Die Knochentypen mit den meisten Funden & Maßen durchgehen:
1. statistische Übersicht für die beiden häufigsten vorhandene Dimension PRO PERIODE
2. Trends der Größenentwicklung über die Perioden zusammenfassen

In [ ]:
# Raincloudplot probieren - seaborn Notation recherchieren?
df_tmp = bone_types["Metatarsal"]["df"]
df_metatarsal_bd = df_tmp.loc[df_tmp['MEASTYPE'] == 'Bd']
df_metatarsal_bd_grp = df_metatarsal_bd.groupby('PERIOD')
x_1 = list(df_metatarsal_bd_grp.get_group('Bronze - Iron Age').MEASURE)
x_2 = list(df_metatarsal_bd_grp.get_group('Roman - Saxon').MEASURE)
x_3 = list(df_metatarsal_bd_grp.get_group('Medieval').MEASURE)
x_4 = list(df_metatarsal_bd_grp.get_group('Modern').MEASURE)

# auf y stehen die Labels für die Perioden, x bildet die Größe ab

In [ ]:
# ggf. für die deskriptive Übersicht? Kein weiterer Nutzen, wollte nur Notation merken
df = bone_types["Metatarsal"]["df"]
df_grp = df.groupby(['MEASTYPE','PERIOD'])
df_grp['MEASURE'].var()

In [ ]:
df_grp['MEASURE'].std()
df_grp['MEASURE'].var()
df_grp['MEASURE'].min()
df_grp['MEASURE'].mean()
df_grp['MEASURE'].median()
df_grp['MEASURE'].max()